In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from numpy.linalg import norm
from numpy import trace
from numpy import identity
from numpy import argpartition
from numpy.linalg import multi_dot
from scipy.linalg import hadamard
from copy import deepcopy

import matplotlib.pyplot as plt

In [ ]:
from pyqalm.data_structures import SparseFactors
from pyqalm.qalm import palm4msa_fast1, PALM4MSA

In [ ]:
def get_data(small_dim=2**10, large_dim=2**11, n_nonzeros_small=None, n_nonzeros_large=None):
    if n_nonzeros_small is None:
        n_nonzeros_small = 2*small_dim
    if n_nonzeros_large is None:
        n_nonzeros_large = 2*large_dim
    n_factors = int(np.log2(small_dim))
    A = []
    for _ in range(n_factors - 1):
        A.append(np.zeros((small_dim, small_dim)))
        A[-1].flat[np.random.choice(small_dim**2, size=n_nonzeros_small)] = np.random.randn(n_nonzeros_small)
    A += [np.zeros((small_dim, large_dim))]
    A[-1].flat[np.random.choice(small_dim * large_dim, size=n_nonzeros_large)] = np.random.randn(n_nonzeros_large)
    P = np.linalg.multi_dot(A)
    S = SparseFactors(A)
    return A, P, S

# Execution time for computing a matrix-vector product

In [ ]:
print('Execution time for computing a matrix-vector product (square case)')
lst_small_dim = 2**np.arange(8, 13)
times = np.empty((len(lst_small_dim), 4))
for i, small_dim in enumerate(lst_small_dim):
    large_dim = small_dim
    print('Dimension:', small_dim, large_dim)
    A, P, S = get_data(small_dim=small_dim, 
                       large_dim=large_dim, 
                       n_nonzeros_small=2*small_dim, 
                       n_nonzeros_large=2*large_dim)
    x = np.random.randn(large_dim)
    t = %timeit -o y = P @ x
    times[i, 0] = t.average
    t = %timeit -o y = S(x)
    times[i, 1] = t.average
    t = %timeit -o y = S @ x
    times[i, 2] = t.average
    t = %timeit -o y = S.matvec(x)
    times[i, 3] = t.average

In [ ]:
plt.loglog(lst_small_dim, times[:, 0], label='dense')
plt.loglog(lst_small_dim, times[:, 1], label='Sparse: call')
plt.loglog(lst_small_dim, times[:, 2], label='Sparse: @')
plt.loglog(lst_small_dim, times[:, 3], label='Sparse: matvec')
plt.legend()
plt.grid()
plt.title('Execution time for computing a matrix-vector product (square case)')

In [ ]:
print('Execution time for computing a matrix-vector product (rectangular case)')
lst_small_dim = 2**np.arange(8, 13)
times = np.empty((len(lst_small_dim), 4))
for i, small_dim in enumerate(lst_small_dim):
    large_dim = small_dim * 4
    print('Dimension:', small_dim, large_dim)
    A, P, S = get_data(small_dim=small_dim, 
                       large_dim=large_dim, 
                       n_nonzeros_small=2*small_dim, 
                       n_nonzeros_large=2*large_dim)
    x = np.random.randn(large_dim)
    t = %timeit -o y = P @ x
    times[i, 0] = t.average
    t = %timeit -o y = S(x)
    times[i, 1] = t.average
    t = %timeit -o y = S @ x
    times[i, 2] = t.average
    t = %timeit -o y = S.matvec(x)
    times[i, 3] = t.average

In [ ]:
plt.loglog(lst_small_dim, times[:, 0], label='dense')
plt.loglog(lst_small_dim, times[:, 1], label='Sparse: call')
plt.loglog(lst_small_dim, times[:, 2], label='Sparse: @')
plt.loglog(lst_small_dim, times[:, 3], label='Sparse: matvec')
plt.legend()
plt.grid()
plt.title('Execution time for computing a matrix-vector product (rectangular case)')

# Execution time for computing the spectral norm

In [ ]:
from scipy.sparse.linalg import eigs, svds
from pyqalm.data_structures import SparseFactors
def get_spectral_norm(S, method='svds'):
    if method == 'svds':
        a = svds(A=S, k=1, return_singular_vectors=False)
        return a[0]
    elif method == 'eigs':
        if S.shape[0] > S.shape[1]:
            SS = SparseFactors(S.adjoint().get_list_of_factors() + S.get_list_of_factors())
        else:
            SS = SparseFactors(S.get_list_of_factors() + S.adjoint().get_list_of_factors())
        a = eigs(A=SS, k=1, return_eigenvectors=False)
        return np.sqrt(np.real(a[0]))

        


In [ ]:
small_dim = 32
large_dim = small_dim * 4
A, P, S = get_data(small_dim=small_dim, 
                       large_dim=large_dim, 
                       n_nonzeros_small=2*small_dim, 
                       n_nonzeros_large=2*large_dim)
a = np.linalg.svd(P, compute_uv=False)
print(a[0])
a = get_spectral_norm(S, method='svds')
print(a)
a = get_spectral_norm(S, method='eigs')
print(a)
a = get_spectral_norm(S.transpose(), method='eigs')
print(a)

print('Square')
large_dim = small_dim
A, P, S = get_data(small_dim=small_dim, 
                       large_dim=large_dim, 
                       n_nonzeros_small=2*small_dim, 
                       n_nonzeros_large=2*large_dim)
print(np.linalg.norm(P, ord=2))
a = np.linalg.svd(P, compute_uv=False)
print(a[0])
a = get_spectral_norm(S, method='svds')
print(a)
a = get_spectral_norm(S, method='eigs')
print(a, np.abs(a), np.abs(a)**2)


In [ ]:
print('Execution time for computing the spectral norm (rectangular case)')
lst_small_dim = 2**np.arange(8, 13)
times = np.empty((len(lst_small_dim), 4))
for i, small_dim in enumerate(lst_small_dim):
    large_dim = small_dim * 4
    print('Dimension:', small_dim, large_dim)
    A, P, S = get_data(small_dim=small_dim, 
                       large_dim=large_dim, 
                       n_nonzeros_small=2*small_dim, 
                       n_nonzeros_large=2*large_dim)
    SH = S.adjoint()
    t = %timeit -o np.linalg.norm(P, ord=2)
    times[i, 0] = t.average
    t = %timeit -o S.compute_spectral_norm(method='svds')
    times[i, 1] = t.average
    t = %timeit -o S.compute_spectral_norm(method='eigs')
    times[i, 2] = t.average
    t = %timeit -o SH.compute_spectral_norm(method='eigs')
    times[i, 3] = t.average

In [ ]:
plt.loglog(lst_small_dim, times[:, 0], label='dense')
plt.loglog(lst_small_dim, times[:, 1], label='Sparse: svds')
plt.loglog(lst_small_dim, times[:, 2], label='Sparse: eigs')
plt.loglog(lst_small_dim, times[:, 3], label='Sparse (Hermitian): eigs')
plt.legend()
plt.grid()
plt.title('Execution time for computing a matrix-vector product (square case)')

# Execution time for computing the product

In [ ]:
print('Execution time for computing the product')
%timeit P = np.linalg.multi_dot(A)
%timeit PP = S.compute_product()